In [1]:
from datasets import load_dataset
import pandas as pd
import evaluate
dataset = load_dataset("IlyaGusev/gazeta")

/home/marie/newenv/lib/python3.8/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (/home/marie/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/e2d171980aa248bc22e0af4f8485ad69071fc8e5f3d54a253c71eb434f6694bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
# ! pip install rouge_score
# ! pip install evaluate

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
})

In [4]:
! pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 KB 445.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 1.5 MB/s eta 0:00:0000:0100:010m
You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.


In [28]:
bleu = evaluate.load("bleu")
from datasets import load_dataset, load_metric
# bleu = load_metric('bleu')

In [6]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [7]:
import torch

In [8]:
device = torch.device('cuda:0')

In [9]:
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [10]:
len(dataset['test']['text'])

6793

In [11]:
len(dataset['train']['text'])

60964

In [12]:
from tqdm import tqdm

In [13]:
for text in tqdm(dataset['test']['text'][:10], total=len(dataset['test']['text'][:10])):

    input_ids = tokenizer(
        text,
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]
    input_ids = input_ids.to(device)
    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    # задача -- для каждого текста посчитать bleu, и вывести среднюю метрику по датасету dataset['test']

  0%|          | 0/10 [00:00<?, ?it/s]/home/marie/newenv/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


In [21]:
prediction = list([summary])

In [16]:
prediction

['Президент США Дональд Трамп назвал «марионеткой» своего соперника на предстоящих выборах демократа Джо Байдена. Если Байден придет к власти, то его быстро отберут радикальные левые силы, которые устроят в Америке революцию, заявил Трамп. Он также подчеркнул, что не желает новой гражданской войны в США — между «леваками» и его сторонниками. Накануне Байден обвинил Трампа в «покорности» перед российскими властями.']

In [26]:
reference = [[dataset['test']['text'][9]]]

In [25]:
reference

['Президент США Дональд Трамп назвал «марионеткой» своего соперника на предстоящих выборах американского лидера, демократа Джо Байдена. Об этом сообщает телеканал Fox News. Если Байден придет к власти, то ее очень быстро отберут радикальные левые силы, которые устроят в Америке революцию, заявил Трамп. «Байден… он слабый человек и был таким всю свою жизнь. Его контролируют, как куклу... это революция. Он не должен баллотироваться в президенты», — отметил он. В ответ на вопрос телеведущей, кто же управляет «марионеткой Байденом», Трамп заявил, что она «этих людей не знает» и вдаваться в подробности не стал. Он также подчеркнул, что не желает новой гражданской войны в США — между «леваками» и его сторонниками. «Этого я не хочу, я оставляю это [столкновения с радикалами] нашим правоохранителям», — добавил Трамп. Накануне Джо Байден обвинил Трампа в «покорности» перед российскими властями. «Мы видели сообщения, что российские силы атаковали американских солдат в Сирии, поранили наших солда

In [29]:
bleu.compute(predictions=prediction, references=reference)

{'bleu': 0.00013714348069350797,
 'precisions': [0.9850746268656716,
  0.9393939393939394,
  0.8615384615384616,
  0.78125],
 'brevity_penalty': 0.00015437599535098924,
 'length_ratio': 0.10229007633587786,
 'translation_length': 67,
 'reference_length': 655}